# Quick start

## 1.  Install Feast
Install the Feast SDK and CLI using pip:

In this tutorial, we focus on a local deployment. For a more in-depth guide on how to use Feast with
Snowflake / GCP / AWS deployments, see Running Feast with [Snowflake/GCP/AWS](https://docs.feast.dev/how-to-guides/feast-snowflake-gcp-aws)

```shell
pip install feast

poetry add feast
```

## 2. Step 2: Create a feature repository
Bootstrap a new feature repository using feast init from the command line.

```shell
feast init feature_repo
cd feature_repo
```

This will create a directory in your current location. In my case, I have created the following directory

```shell
/home/pliu/git/FeatureEngineering/feature_stores/01.Feast/01.QuickStart/feature_repo

# when you open it, it has the following contents
├── data
│   └── driver_stats.parquet
├── example.py
├── feature_store.yaml
└── __init__.py

```

- data/ contains raw demo parquet data
- example.py contains demo feature definitions
- feature_store.yaml contains a demo setup configuring where data sources are

### Explore feature_store.yaml
Below is the content of the auto generated feature_store.yaml

```yaml
project: feature_repo
registry: data/registry.db
provider: local
online_store:
    path: data/online_store.db
```

The most import config is the **provider**. This defines where the raw data exists
(for generating training data & feature values for serving), and where to materialize feature values to in the
online store (for serving).

Valid values for provider in feature_store.yaml are:
- local: use file source with SQLite/Redis
- gcp: use BigQuery/Snowflake with Google Cloud Datastore/Redis
- aws: use Redshift/Snowflake with DynamoDB/Redis

Note that there are many other sources Feast works with, including `Azure, Hive, Trino, and PostgreSQL` via community
plugins. See [Third party integrations](https://docs.feast.dev/getting-started/third-party-integrations) for all supported datasources.

A custom setup can also be made by following [adding a custom provider](https://docs.feast.dev/how-to-guides/creating-a-custom-provider).

### Inspecting raw data

The raw feature data we have in this demo is stored in a local parquet file. The dataset captures hourly stats of a driver in a ride-sharing app.

In [1]:
import pandas as pd

data_path="feature_repo/data/driver_stats.parquet"
df=pd.read_parquet(data_path)

df.head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2022-06-07 13:00:00+00:00,1005,0.913727,0.034992,655,2022-06-22 13:50:22.688
1,2022-06-07 14:00:00+00:00,1005,0.508678,0.651014,38,2022-06-22 13:50:22.688
2,2022-06-07 15:00:00+00:00,1005,0.896986,0.741025,788,2022-06-22 13:50:22.688
3,2022-06-07 16:00:00+00:00,1005,0.189035,0.729997,894,2022-06-22 13:50:22.688
4,2022-06-07 17:00:00+00:00,1005,0.271720,0.254235,91,2022-06-22 13:50:22.688


## Step 3: Register feature definitions and deploy your feature store

The apply command scans python files in the current directory for feature view/entity definitions, registers the objects, and deploys infrastructure. In this example, it reads example.py (shown again below for convenience) and sets up SQLite online store tables. Note that we had specified SQLite as the default online store by using the local provider in feature_store.yaml.